In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('law_data.csv')

#Preprocessing
df = df.drop(columns = ['Unnamed: 0'])
df = df[df['region_first'] != 'PO']

#Code race
race_coded = pd.get_dummies(df['race'])
df = pd.concat([df, race_coded],axis=1)

#Code gender
gender_coded = pd.get_dummies(df['sex'])
gender_coded.columns = ['female', 'male']
df = pd.concat([df, gender_coded],axis=1)

df = df.drop(columns = ['race', 'sex'])
sense_cols = ['Amerindian', 'Asian', 'Black', 'Hispanic', 'Mexican', 'Other','Puertorican', 'White', 'female', 'male']

In [3]:
from sklearn.model_selection import train_test_split

X = df.loc[:, df.columns !='ZFYA']
y = df['ZFYA']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [4]:
n = len(X_train)
ne = len(X_test)
K = len(sense_cols)  #latent variable knowledge that affects gpa, last and fya, but is not related to race and sex

In [5]:
X_train['LSAT'] = X_train['LSAT'].astype(int)
X_test['LSAT'] = X_test['LSAT'].astype(int)

In [6]:
law_stan_data = {
    'N' : n, #number of observations
    'K' : K, 
    'a' : np.array(X_train[sense_cols]), #protected variable race and sex
    'ugpa' : np.array(X_train['UGPA']), 
    'lsat' : np.array(X_train['LSAT']),
    'zfya' : np.array(y_train)
       }

In [7]:
model= """
data {
  int<lower = 0> N; // number of observations
  int<lower = 0> K; // number of covariates
  matrix[N, K]   a; // sensitive variables
  real           ugpa[N]; // UGPA
  int            lsat[N]; // LSAT
  real           zfya[N]; // ZFYA
  
}

transformed data {
  
 vector[K] zero_K;
 vector[K] one_K;
 
 zero_K = rep_vector(0,K);
 one_K = rep_vector(1,K);

}

parameters {

  vector[N] u;

  real ugpa0;
  real eta_u_ugpa;
  real lsat0;
  real eta_u_lsat;
  real eta_u_zfya;
  
  vector[K] eta_a_ugpa;
  vector[K] eta_a_lsat;
  vector[K] eta_a_zfya;
  
  
  real<lower=0> sigma_g_Sq;
}

transformed parameters  {
 // Population standard deviation (a positive real number)
 real<lower=0> sigma_g;
 // Standard deviation (derived from variance)
 sigma_g = sqrt(sigma_g_Sq);
}

model {
  
  // don't have data about this
  u ~ normal(0, 1);
  
  ugpa0 ~ normal(0, 1);
  eta_u_ugpa ~ normal(0, 1);
  lsat0 ~ normal(0, 1);
  eta_u_lsat ~ normal(0, 1);
  eta_u_zfya ~ normal(0, 1);

  eta_a_ugpa ~ normal(zero_K, one_K);
  eta_a_lsat ~ normal(zero_K, one_K);
  eta_a_zfya ~ normal(zero_K, one_K);

  sigma_g_Sq ~ inv_gamma(1, 1);

  // have data about these
  ugpa ~ normal(ugpa0 + eta_u_ugpa * u + a * eta_a_ugpa, sigma_g);
  lsat ~ poisson(exp(lsat0 + eta_u_lsat * u + a * eta_a_lsat));
  zfya ~ normal(eta_u_zfya * u + a * eta_a_zfya, 1);

}
"""

u -> K
a -> Include all sensitive variables sex and race using real data

In [8]:
import stan
import nest_asyncio
nest_asyncio.apply()

posterior = stan.build(model, data=law_stan_data, random_seed=1)
fit = posterior.sample(num_chains= 1, num_samples= 50)

Building...



Building: found in cache, done.Messages from stanc:
Sampling:   0%
Sampling:   0% (1/1050)
Sampling:  10% (100/1050)
Sampling:  19% (200/1050)
Sampling:  29% (300/1050)
Sampling:  38% (400/1050)
Sampling:  48% (500/1050)
Sampling:  57% (600/1050)
Sampling:  67% (700/1050)
Sampling:  76% (800/1050)
Sampling:  86% (900/1050)
Sampling:  95% (1000/1050)
Sampling:  95% (1001/1050)
Sampling: 100% (1050/1050)
Sampling: 100% (1050/1050), done.
Messages received during sampling:
  Gradient evaluation took 0.008761 seconds
  1000 transitions using 10 leapfrog steps per transition would take 87.61 seconds.
  Adjust your expectations accordingly!


In [9]:
U_train = fit['u'].mean(axis = 1)
pd.DataFrame(U_train).to_csv('U_train.csv')

ugpa0 = fit['ugpa0'].mean()
eta_u_ugpa = fit['eta_u_ugpa'].mean()
eta_a_ugpa = fit['eta_a_ugpa'].mean(axis = 1)

lsat0 = fit['lsat0'].mean()
eta_u_lsat = fit['eta_u_lsat'].mean()
eta_a_lsat = fit['eta_a_lsat'].mean(axis = 1)

sigma_g = fit['sigma_g'].mean()

In [10]:
model_u = """
data {
  int<lower = 0> N; // number of observations
  int<lower = 0> K; // number of covariates
  
  matrix[N, K]   a; // sensitive variables
  real           ugpa[N]; // UGPA
  int            lsat[N]; // LSAT
  //real           zfya[N]; // ZFYA
  //int<lower = 0> pass[N]; // PASS
  real           ugpa0;
  real           eta_u_ugpa;
  vector[K]      eta_a_ugpa;
  real           lsat0;
  real           eta_u_lsat;
  vector[K]      eta_a_lsat;
  //real           eta_u_zfya;
  //vector[K]      eta_a_zfya;
  //real           pass0;
  //real           eta_u_pass;
  //vector[K]      eta_a_pass;
  real           sigma_g;
  
 
}


parameters {

  vector[N] u;

}


model {
  
  u ~ normal(0, 1);

  // have data about these
  ugpa ~ normal(ugpa0 + eta_u_ugpa * u + a * eta_a_ugpa, sigma_g); 
  lsat ~ poisson(exp(lsat0 + eta_u_lsat * u + a * eta_a_lsat)); 
  //zfya ~ normal(eta_u_zfya * u + a * eta_a_zfya,1);
  //pass ~ bernoulli_logit(pass0 + eta_u_pass * u + a * eta_a_pass);
  
}
"""

In [11]:
law_stan_test_data = {
    'N' : ne,
    'K' : K,
    'a' : np.array(X_test[sense_cols]), #protected variable race and sex
    'ugpa' : np.array(X_test['UGPA']), 
    'lsat' : np.array(X_test['LSAT']),
    'ugpa0' : ugpa0,
    'eta_u_ugpa' : eta_u_ugpa,
    'eta_a_ugpa' : eta_a_ugpa,
    'lsat0' : lsat0, 
    'eta_u_lsat' : eta_u_lsat,
    'eta_a_lsat' : eta_a_lsat,
    'sigma_g' : sigma_g    
}

In [12]:
posterior_test = stan.build(model_u, data=law_stan_test_data, random_seed=1)
fit_test = posterior_test.sample(num_chains= 1, num_samples=50)

Building...



Building: found in cache, done.Sampling:   0%
Sampling:   0% (1/1050)
Sampling:  10% (100/1050)
Sampling:  19% (200/1050)
Sampling:  29% (300/1050)
Sampling:  38% (400/1050)
Sampling:  48% (500/1050)
Sampling:  57% (600/1050)
Sampling:  67% (700/1050)
Sampling:  76% (800/1050)
Sampling:  86% (900/1050)
Sampling:  95% (1000/1050)
Sampling:  95% (1001/1050)
Sampling: 100% (1050/1050)
Sampling: 100% (1050/1050), done.
Messages received during sampling:
  Gradient evaluation took 0.001374 seconds
  1000 transitions using 10 leapfrog steps per transition would take 13.74 seconds.
  Adjust your expectations accordingly!


In [14]:
U_test = fit_test['u'].mean(axis = 1)

# Fair Model

In [15]:
# 2. Fair + non-deterministic model (deterministic)
from sklearn.linear_model import LinearRegression

X_fair = U_train.reshape(-1,1)
X_fair_test = U_test.reshape(-1, 1)

lr_fair = LinearRegression()
lr_fair.fit(X_fair, y_train)

LinearRegression()

# Unfair models


In [18]:
#A. Full model
full_list = ['LSAT', 'UGPA', 'Amerindian', 'Asian', 'Black', 'Hispanic', 'Mexican','Other', 'Puertorican', 'White', 'female', 'male']
lr_full = LinearRegression()
lr_full.fit(X_train[full_list], y_train)

#B. Unaware model
unaware = ['LSAT', 'UGPA']
lr_unaware = LinearRegression()
lr_unaware.fit(X_train[unaware], y_train)


LinearRegression()

In [21]:
#Compile to one dataset
X_train['ZFYA'] = y_train
X_train['Knowledge'] = U_train

X_train['Init_class'] = np.sign(X_train['ZFYA'])

X_train['Fair_pred'] = lr_fair.predict(X_fair)
X_train['Fair_pred_class'] = np.sign(X_train['Fair_pred'])

X_train['full_pred'] = lr_full.predict(X_train[full_list])
X_train['full_pred_class'] = np.sign(X_train['full_pred'])

X_train['unaware_pred'] = lr_unaware.predict(X_train[unaware])
X_train['unaware_pred_class'] = np.sign(X_train['unaware_pred'])

X_train.to_csv('Full_Training_df.csv')

In [22]:
X_test['ZFYA'] = y_test
X_test['Knowledge'] = U_test

X_test['Init_class'] = np.sign(X_test['ZFYA'])

X_test['Fair_pred'] = lr_fair.predict(X_fair_test)
X_test['Fair_pred_class'] = np.sign(X_test['Fair_pred'])

X_test['full_pred'] = lr_full.predict(X_test[full_list])
X_test['full_pred_class'] = np.sign(X_test['full_pred'])

X_test['unaware_pred'] = lr_unaware.predict(X_test[unaware])
X_test['unaware_pred_class'] = np.sign(X_test['unaware_pred'])

X_test.to_csv('Full_Test_df.csv')

# Metrics